In [1]:
pip install duckdb

  Obtaining dependency information for duckdb from https://files.pythonhosted.org/packages/99/61/1ef0d3d13908b5ae6637103e0dbeafc58028f662caa366ecd871154dbc9f/duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (763 bytes)
Using cached duckdb-0.10.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.8 MB)
Note: you may need to restart the kernel to use updated packages.


### ZipCode info

In [13]:
import duckdb
duckdb_conn = duckdb.connect(database=':memory:')
duckdb_cursor = duckdb_conn.cursor()
import pandas as pd
df_states_zip = pd.read_csv('df_states_zip.csv')
df_states_zip = df_states_zip[['zip_code','city','state','county','timezone','lat','lon']]

query = "CREATE TABLE states_info (\
zip_code INT64 PRIMARY KEY,\
city VARCHAR,\
state VARCHAR,\
county VARCHAR,\
timezone VARCHAR,\
lat FLOAT,\
lon FLOAT);\
"
duckdb_cursor.execute(query)


query_insert = """
INSERT INTO states_info VALUES (?, ?, ?, ?, ?, ?, ?);
"""
data_tuples = [tuple(row) for row in df_states_zip.values]
duckdb_cursor.executemany(query_insert, data_tuples)
duckdb_conn.commit()

query_select = "SELECT * FROM states_info"
duckdb_cursor.execute(query_select)

# Fetch and print the results
result = duckdb_cursor.fetchall()
for row in result[:10]:
    print(row)
duckdb_cursor.close()
duckdb_conn.close()

(35004, 'Moody', 'AL', 'St. Clair County', 'America/Chicago', 33.60340118408203, -86.49440002441406)
(35005, 'Adamsville', 'AL', 'Jefferson County', 'America/Chicago', 33.592899322509766, -86.99400329589844)
(35006, 'Adger', 'AL', 'Jefferson County', 'America/Chicago', 33.44620132446289, -87.2229995727539)
(35007, 'Alabaster', 'AL', 'Shelby County', 'America/Chicago', 33.21870040893555, -86.78350067138672)
(35010, 'Alexander City', 'AL', 'Tallapoosa County', 'America/Chicago', 32.901100158691406, -85.91780090332031)
(35011, 'Alexander City', 'AL', 'Tallapoosa County', 'America/Chicago', 32.93939971923828, -85.94660186767578)
(35013, 'Allgood', 'AL', 'Blount County', 'America/Chicago', 33.91189956665039, -86.5093994140625)
(35014, 'Alpine', 'AL', 'Talladega County', 'America/Chicago', 33.34519958496094, -86.26570129394531)
(35015, 'Alton', 'AL', 'Jefferson County', 'America/Chicago', 33.579498291015625, -86.63760375976562)
(35016, 'Arab', 'AL', 'Marshall County', 'America/Chicago', 34.3

In [4]:
df_states_zip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41417 entries, 0 to 41416
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  41417 non-null  int64  
 1   zip_code    41417 non-null  int64  
 2   city        41417 non-null  object 
 3   state       41417 non-null  object 
 4   county      41363 non-null  object 
 5   timezone    41413 non-null  object 
 6   area_codes  41417 non-null  object 
 7   lat         41417 non-null  float64
 8   lon         41417 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 2.8+ MB


### sales tax

In [19]:
import duckdb
duckdb_conn = duckdb.connect(database=':memory:')
duckdb_cursor = duckdb_conn.cursor()

import psycopg2
host="13.43.84.151"
dbname="postgres"
user="ping-fan.chen.23@ucl.ac.uk"
password="khsLcR"
port="5432"
schema="schema_pingfanchen23uclacuk"
conn = psycopg2.connect(
    host=host,
    database=dbname,
    user=user,
    password=password,
    port=port
)
cur = conn.cursor()
cur.execute("SELECT * FROM schema_pingfanchen23uclacuk.sale_tax_by_zip;")
data = cur.fetchall()

query = "CREATE TABLE sales_tax (\
zip_code INT64 PRIMARY KEY FOREIGN KEY REFERENCES states_info(zip_code),\
additional_rate FLOAT,\
city_rate FLOAT,\
county_rate FLOAT,\
state_rate FLOAT,\
total_rate FLOAT\
);\
"
duckdb_cursor.execute(query)

query_insert = """
INSERT INTO sales_tax VALUES (?, ?, ?, ?, ?, ?);"""

duckdb_cursor.executemany(query_insert, data)
duckdb_conn.commit()

query_select = "SELECT * FROM sales_tax"
duckdb_cursor.execute(query_select)

# Fetch and print the results
result = duckdb_cursor.fetchall()
print(result[:10])
duckdb_cursor.close()
duckdb_conn.close()
cur.close()
conn.close()

[(0, 0.03999999910593033, 0.0, 0.03999999910593033, 0.10000000149011612, 35004.0), (0, 0.03999999910593033, 0.0, 0.03999999910593033, 0.10000000149011612, 35173.0), (0, 0.029999999329447746, 0.029999999329447746, 0.03999999910593033, 0.10000000149011612, 35444.0), (0, 0.04500000178813934, 0.009999999776482582, 0.03999999910593033, 0.0949999988079071, 35634.0), (0, 0.03999999910593033, 0.009999999776482582, 0.03999999910593033, 0.09000000357627869, 35962.0), (0, 0.03500000014901161, 0.02500000037252903, 0.03999999910593033, 0.10000000149011612, 36112.0), (0, 0.03500000014901161, 0.02500000037252903, 0.03999999910593033, 0.10000000149011612, 36420.0), (0, 0.0, 0.009999999776482582, 0.03999999910593033, 0.05000000074505806, 36583.0), (0, 0.03999999910593033, 0.009999999776482582, 0.03999999910593033, 0.09000000357627869, 36849.0), (0, 0.03999999910593033, 0.0, 0.0, 0.03999999910593033, 99585.0)]


### Median rental price

In [92]:
def reshape_table(column,df):
    table = df[column].reset_index()
    city,state = column[0].split(',')
    table['State'] = state
    table['City']=city
    table['Price'] = table[column]
    table['Time'] = table['index']
    table = table[['State','City','Time','Price']]
    return table

In [93]:
import pandas as pd
df_median = pd.read_csv('median_rental_price.csv',skiprows=0)
df_median = df_median.loc[1:,df_median.columns[~df_median.columns.isin(['RegionID','SizeRank','RegionType'])]]
# df_median.groupby('StateName', group_keys=True)[df_median.columns].apply(lambda x: x)
df = pd.DataFrame(df_median[df_median.columns[2:]].values.T,index = df_median.columns[2:],columns= [df_median['RegionName'].values])
df.reset_index(names='Time').head()
lst = []
for column in df.columns[1:]:
    lst.append(reshape_table(column,df))

In [102]:
# df = pd.concat(lst)
# df.columns = ['State','City','Time','Price']
df['State'].astype('str')
df['City'].astype('str')
df.dropna(subset = ['Price'],inplace=True)

In [117]:
df.info()
df.to_csv('median_rent_price.csv',index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 104977 entries, 0 to 190
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   State   104977 non-null  object 
 1   City    104977 non-null  object 
 2   Time    104977 non-null  object 
 3   Price   104977 non-null  float64
dtypes: float64(1), object(3)
memory usage: 4.0+ MB


In [118]:
import duckdb
duckdb_conn = duckdb.connect(database=':memory:')
duckdb_cursor = duckdb_conn.cursor()
import pandas as pd
df_median = pd.read_csv('median_rent_price.csv')

query = "CREATE TABLE median_rental_price (\
State VARCHAR FOREIGN KEY REFERENCES states_info(state),\
City VARCHAR FOREIGN KEY REFERENCES states_info(city),\
Time DATETIME,\
Price FLOAT);\
"
duckdb_cursor.execute(query)


query_insert = """
INSERT INTO median_rental_price VALUES (?, ?, ?, ?);
"""
duckdb_cursor.executemany(query_insert, df_median.values.tolist())
duckdb_conn.commit()

query_select = "SELECT * FROM median_rental_price"
duckdb_cursor.execute(query_select)

# Fetch and print the results
result = duckdb_cursor.fetchall()
for row in result[:10]:
    print(row)
duckdb_cursor.close()
duckdb_conn.close()

(' CA', 'Los Angeles', datetime.datetime(2008, 2, 29, 0, 0), 470000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 3, 31, 0, 0), 456000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 4, 30, 0, 0), 457000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 5, 31, 0, 0), 440000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 6, 30, 0, 0), 435000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 7, 31, 0, 0), 420000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 8, 31, 0, 0), 402250.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 9, 30, 0, 0), 379000.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 10, 31, 0, 0), 370050.0)
(' CA', 'Los Angeles', datetime.datetime(2008, 11, 30, 0, 0), 350000.0)
